In [ ]:

# HMMクラス
# 숫자 계산 모듈 (numpy) 가져 오기
import numpy as np
# json 형식의 입출력을 수행하는 모듈 가져 오기
import json

import sys

class MonoPhoneHMM():
    def __init__(self):
        # 음소 리스트
        self.phones = []
        # 음소수
        self.num_phones = 1
        # 각 음소 HMM의 상태 수
        self.num_states = 1
        # GMM 혼합 수
        self.num_mixture = 1
        # 특징량 벡터의 차원 수
        self.num_dims = 1
        # 정규 분포(Single Gaussian Model: SGM)
        # 매개 변수
        self.pdf = None
        # 전이 확률 (대수)
        self.trans = None
        # log(0)의 근사값 
        self.LZERO = -1E10
        # 확률 계산에 추가되는 최소값
        # 계산량을 줄이기 위해서 값보다 작은 확률은
        # 일부 계산에서 무시됨
        self.MINVAR = 1E-4
        # 학습시/인식시에 사용하는 파라미터
        self.elem_prob = None
        # 각 상태에 대해 계산되는 로그 확률
        self.state_prob = None
        #전향확률
        self.alpha = None
        # 후향확률
        self.beta = None
        # HMM 우도
        self.loglikelihood = 0
        # 매개변수 업데이트를 위한 변수
        self.pdf_accmulators = None
        self.trans_accumulators = None
        # 비터비 알고리즘에 사용되는 누적 확률
        self.score = None
        # 비타비 경로를 기억하는 행렬
        self.track = None
        # 비터비 알고리즘으로 점수
        self.viterbi_score = None
        
    def make_proto(self, phone_list, num_states, prob_loop, num_dims):
        '''
        HMM프로토타입을 생성
        phone_list : 음소목록
        num_states : 각 음소의 HMM 상태수
        prob_loop: 자기루프 확률
        num_dims: 특징값의 차원 수 
        '''
        # 음소 리스트 획득
        self.phones = phone_list
        # 음소 수 획득
        self.num_phones = len(self.phones)
        # 각 음소 HMM의 상태 수를 얻습니다.
        self.num_states = num_states
        # 특징 벡터의 차원 수를 얻습니다.
        self.num_dims = num_dims
        # GMM의 혼합 수는 1입니다.
        self.num_mixture = 1
        # 정규 분포 만들기
        # 음소 번호 p, 상태 번호 s, 혼합 요소 번호 m
        # 의 정규분포는 pdf[p][s][m]로 액세스한다
        # pdf[p][s][m] = gaussian
        self.pdf = []
        
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                tmp_s =[]
                for m in range(self.num_mixture):
                    mu = np.zeros(self.num_dims)
                    # 평균값 벡터 정의
                    var = np.ones(self.num_dims)
                    # 혼합 수는 1이므로 혼합 가중치는 1.0
                    weight = 1.0
                    # gConst 항 계산
                    gconst = self.calc_gconst(var)
                    # 정규 분포를 사전 형식으로 정의
                    gaussian = {'weight': weight, 'mu':mu, 'var':var,'gConst':gconst}
                    # 정규 분포 추가
                    temp_s.append(gaussian)
                tmp_p.append(tmp_s)
            self.pdf.append(tmp_p)
            
        # 상태 전이 확률 (의 대수 값) 생성
        # 음소 번호 p, 상태 번호 s의 전이 확률은
        # trans[p][s] = [loop, next]
        # loop: 자기 루프 확률
        # next : 다음 상태로의 전환 확률
        
        prob_next = 1.0 - prob_loop
        
        log_prob_loop = np.log(prob_loop) if prob_loop > self.ZERO else self.LZERO
        log_prob_next = np.log(prob_next) if prob_next > self.ZERO else self.LZERO
        
        # self.trans에 저장
        self.trans = []
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                tmp_trans = np.array([log_prob_loop, log_prob_next])
                tmp_p.append(tmp_trans)
            self.trans.append(tmp_p)
            
    def calc_gconst(self, variance):
        '''gConst 항 (정규 분포의 상수 항
            대수 값) 계산
        variance: 대각 공분산 행렬의 대각 성분
        '''
        gconst = self.num_dims * np.log(2.0 * np.pi) + np.sum(np.log(variance))
        return gconst
    
    def calc_pdf(self, pdf, obs):
        '''지정된 정규 분포에서 로그 우도 계산
        pdf: 정규 분포
        obs: 입력 특징량
                 1프레임분의 벡터에서도
                 프레임 x 차원의 배열에서도 입력 가능
        logprob: 로그 우도
                 1프레임분을 주었을 경우는 스칼라치
                 여러 프레임 분을 주면
                 프레임 몇 분 크기의 벡터
        '''
        # 상수 항을 제외한 부분 계산 (exp(*) 부분)
        tmp = (obs - pdf['mu'])**2 / pdf['var']
        if np.ndim(tmp) == 2:
            # obs가 [프레임 x 차원] 배열로 입력되면
            tmp = np.sum(tmp, 1)
        elif np.ndim(tmp) == 1:
            # obs가 1 프레임 분의 벡터로 입력되면
            tmp = np.sum(tmp)
        # 상수 항을 붙여 -0.5를 곱한다.
        logprob = -0.5 * (tmp +pdf['gConst'])
        return logprob
    
    def logadd(self, x, y):
        ''' x = log (a) 및 y = log (b)
            log(a+b) 계산
        x: log(a)
        y: log(b)
        z: log(a+b)
        '''
        if x > y :
            z = x + np.log(1.0 + np.exp(y-x))
        else:
            z = y + np.log(1.0 + np.exp(x-y))
        return z
    
    def flat_init(self, mean, var):
        '''플랫 스타트로 초기화
            학습 데이터 전체의 평균 분산
            HMM의 전체 정규 분포 매개 변수로 설정
        mean: 전체 학습 데이터의 평균 벡터
        var : 전체 학습 데이터의 대각 공분산
        '''
        # 차원 수가 일치하지 않으면 오류
        if self.num_dims != len(mean) or self.num_dims !=len(var):
            sys.stderr.write('flat init: invalid mean or var\n')
            return 1
        for p in range(self.num_phones):
            for s in range(self.num_states):
                for m in range(self.num_mixture):
                    pdf = self.pdf[p][s][m]
                    pdf['mu'] = mean
                    pdf['var'] = var
                    pdf['gConst'] = self.calc_gconst(var)
                    
    def calc_out_prob(self, feat, label):
        '''출력 확률 계산
        feat: 1 발화분의 특징량 [프레임수 x 차원수]
        label 1 발화분의 라벨
        '''
        # 특징 량의 프레임 수를 얻는다.
        feat_len = np.shape(feat)[0]
        # 라벨의 길이를 얻는다.
        label_len = len(label)
        # 정규 분포마다 계산되는 로그 확률
        self.elem_prob = np.zeros((label_len, self.num_states, self.num_mixture, feat_len))
        
        # 각 상태 (q, s)에서 시간 t의 출력 확률
        # (state_prob = sum(weight*elem_prob))
        self.state_prob = np.zeros((label_len, self.num_states, feat_len))
        
        # elem_prob, state_prob 계산하기
        # l : 라벨에 몇 번째 음소
        # p : l이 음소 목록의 어느 음소인지
        # s: 상태
        # t: 프레임
        # m: 혼합 요소
        for l,p in enumerate(label):
            for s in range(self.num_states):
                # state_prob를 log(0)로 초기화
                self.state_prob[l][s][:] = self.LZERO * np.ones(feat_len)
                for m in range(self.num_mixture):
                    # 정규 분포 추출
                    pdf = self.pdf[p][s][m]
                    # 확률 계산
                    self.elem_prob[l][s][m][:] = self.calc_pdf(pdf, feat)
                    # GMM 가중치 추가
                    tmp_prob = np.log(pdf['weight']) + self.elem_prob[l][s][m][:]
                    # 확률을 더한다
                    for t in range(feat_len):
                        self.state_prob[l][s][t] = self.logadd(self.state_prob[l][s][t], tmp_prob[t])
    
    def calc_alpha(self, label):
        '''전향 확률 알파 찾기
            left-to-right형 HMM을 전제로 한 구현에
            되어 있다
        label: 라벨
        '''
        # 레이블 길이와 프레임 수를 얻습니다.
        (lebel_len, _, feat_len) = np.shape(self.state_prob)
        # alpha를 log(0)로 초기화
        self.alpha = self.LZERO * np.ones((label_len, self.num_states, feat_len))
        # t = 0 일 때,
        # 항상 첫 음소의 첫 번째 상태에 있음
        self.alpha[0][0][0] = self.state_prob[0][0][0]
        # t: 프레임
        # l : 라벨에 몇 번째 음소
        # p : l이 음소 목록의 어느 음소인지
        # s: 상태
        for t in range(1, feat_len):
            for l in range(0, label_len):
                p = label[l]
                for s in range(0, self.num_states):
                    # 자기 루프 고려
                    self.alpha[l][s][t] = self.alpha[l][s][t-1] + self.trans[p][s][0]
                    if s > 0:
                    # 선행 상태가 아니면,
                    # 이전 상태에서의 전환을 고려
                    tmp = self.alpha[l][s-1][t-1] + self.trans[p][s-1][1]
                    if tmp > self.LSMALL:
                        self.alpha[l][s][t] = self.logadd(self.alpha[l][s][t], tmp)
                        
                    elif l>0:
                        # 첫 번째 음소가 아닌,
                        # 그리고 선두 상태의 경우
                        # 이전 음소의 종단 상태에서 전이
                        prev_p = label[l-1]
                        tmp = self.alpha[l-1][-1][t-1] + self.trans[prev_p][-1][1]
                        # 자기 루프와의 합계 계산
                        if tmp > self.LSMALL:
                            self.alpha[l][s][t] = self.logadd(self.alpha[l][s][t], tmp)
                    # else:
                    # # 선두의 음소이고 선두의 상태의 경우
                    # # 자기 루프 이외의 전환은 없습니다.
                    
                    # state_prob 추가
                    self.alpha[l][s][t] += self.state_prob[l][s][t]
        self.loglikelihood = self.alpha[-1][-1][-1]
    
    def calc_beta(self, label):
        '''후향 확률 beta를 찾는다.
            left-to-right형 HMM을 전제로 한 구현에
            되어 있다
        label: 라벨
        '''
        # 레이블 길이와 프레임 수를 얻습니다.
        (label_len, _, feat_len) = np.shape(self.state_prob)
        # alpha를 log(0)로 초기화
        self.beta = self.LZERO * np.ones((label_len, self.num_states, feat_len))
        # t = -1 (마지막 프레임) 일 때,
        # 항상 마지막 음소의 마지막 상태에 있음
        # (확률은 log(1) = 0)
        self.beta[-1][-1][-1] = 0.0
        # t: 프레임
        # l : 라벨에 몇 번째 음소
        # p : l이 음소 목록의 어느 음소인지
        # s: 상태
        # calc_alpha와 달리 t는 feat_len-2에서 0으로
        # 앞으로의 점에 주의
        for t in range(0, feat_len-1)[::-1]:
            for l in range(0, label_len):
                p = label[l]
                for s in range(0, self.num_states):
                    # 자기 루프 고려
                    self.beta[l][s][t] = self.beta[l][s][t+1] + self.trans[p][s][0] + self.state_prob[l][s][t+1]
                    if s< self.num_states -1:
                    # 종료 상태가 아니면,
                    # 하나의 상태로의 전환을 고려하십시오.    
                    tmp = self.beta[l][s+1][t+1] + self.trans[p][s][1] + self.state_prob[l][s+1][t+1]
                    # 자기 루프와의 합계 계산
                    if tmp > self.LSMALL:
                        self.beta[l][s][t] = self.logadd(self.beta[l][s][t], tmp)
                    
                    elif 1 < label_len -1:
                    # 종단의 음소가 아니라,
                    # 그리고 종단 상태의 경우
                    # 한 후 음소의 선두 상태로의 전환  
                        tmp = self.beta[l+1][0][t+1] + self.trans[p][s][1] + self.state_prob[l+1][0][t+1]
                        
                        if tmp > self.LSMALL:
                            self.beta[l][s][t] = self.logadd(self.beta[l][s][t], tmp)
                    # else:
                    # # 종단의 음소 및 종단 상태의 경우
                    # # 자기 루프 이외의 전환은 없습니다.     
    def reset_accumulators(self):
        '''accumulators (매개 변수 업데이트에 필요한 변수)
            초기화
        '''
        # GMM 업데이트를 위한 accumulators
        self.pdf_accumulators = []
        for p in  range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                tmp_s = []
                for m in range(self.num_mixture):
                    pdf_stats = {}
                    pdf_stats['weight'] = {'num' : self.LZERO, 'den': self.LZERO}
                    pdf_stats['mu'] = {'num' : np.zeros(self.num_dims), 'den': self.LZERO}
                    pdf_stats['var'] = {'num': np.zeros(self.num_dims), 'den': self.LZERO}
                    tmp_s.append(pdf_stats)
                tmp_p.append(tmp_s)
            self.pdf_accumulators.append(tmp_p)
        # 전이 확률을 업데이트하는 accumulators
        self.trans_accumulators = []
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                trans_stats = {'num': np.ones(2) * self.LZERO, 'den': self.LZERO }
                tmp_p.append(trans_stats)
            self.trans_accumulators.append(tmp_p)
        
    def update_accumulators(self, feat, label):
        '''accumulators 업데이트
            left-to-right를 전제로 한 구현이 되고 있다
        feat: 특징량
        label: 라벨
        '''
        # 라벨 길이 가져 오기
        label_len = len(label)
        # 프레임 수 얻기
        feat_len = np.shape(feat)[0]
        
        # t: 프레임
        # l : 라벨에 몇 번째 음소
        # p : l이 음소 목록의 어느 음소인지
        # s: 상태
        for t in range(feat_len):
            for l in range(label_len):
                p = label[l]
                for s in range(self.num_states):
                    if t == 0 and l == 0 and s == 0:
                       # t=0일 때는 반드시 선두 상태
                        # (대수 확률이므로 log(1)=0)
                        lconst = 0
                    elif t == 0 :
                        # t = 0으로 선두 상태가 아닌 경우는
                        # 확률 0이므로 건너 뛰기
                        continue
                    elif s > 0:
                        # t> 0으로 선두 상태가 아닌 경우
                        # 자기 루프
                        lconst = self.alpha[l][s][t-1] + self.trans[p][s][0]
                        # 이전 상태에서의 전환을 고려
                        tmp = self.alpha[l][s-1][t-1] + self.trans[p][s-1][1]
                        # 자기 루프와의 합계 계산
                        if tmp > self.LSMALL:
                            lconst = self.logadd(lconst, tmp)
                    elif l < 0:
                        # t> 0 첫 번째 음소가 아닌,
                        # 그리고 선두 상태의 경우
                        # 자기 루프
                        lconst = self.alpha[l][s][t-1] + self.trans[p][s][0]
                        # 이전 음소의 종단 상태에서 전이
                        prev_p = label[l-1]
                        tmp = self.alpha[l-1][-1][t-1] + self.trans[prev_p][-1][1]
                        # 자기 루프와의 합계 계산
                        if tmp > self.LSMALL:
                            lconst = self.logadd(lconst, tmp)
                    else:
                       # 선두의 음소이고 선두의 상태의 경우
                        # 자체 루프 전용
                        lconst = self.alpha[l][s][t-1] + self.trans[p][s][0]
                    # 뒤로 확률과 1/P 추가
                    lconst += self.beta[l][s][t] - self.loglikelihood
                    # accumulators 업데이트
                    for m in range(self.num_mixture):
                        pdf = self.pdf[p][s][m]
                        L = lconst + np.log(pdf['weight']) + self.elem_prob[l][s][m][t]
                    
                        pdf_accum = self.pdf_accumulators[p][s][m]
                        # 평균값 벡터 갱신식의 분자는
                        # 로그를 취하지 않음
                        pdf_accum['mu']['num'] += np.exp(L) * feat[t]
                        # 분모는 로그에서 업데이트
                        if L > self.LSMALL:
                            pdf_accum['mu']['den'] = self.logadd(pdf_accum['mu']['den'], L)
                        # 대각 공분산 갱신식의 분자는
                        # 로그를 취하지 않음
                        dev = feat[t] - pdf['mu']
                        pdf_accum['var']['num'] += np.exp(L) * (dev**2)
                        # 분모는 평균값과 동일한 값입니다.
                        pdf_accum['var']['den'] = pdf_accum['mu']['den']
                        # GMM 가중치 업데이트 공식의 분자는
                        # 평균 및 분산 분모와 동일한 값
                        pdf_accum['weight']['num'] = pdf_accum['mu']['den']
        
        # 전이 확률 accumulators
        # GMM 가중치 accumulators 분모 업데이트
        for t in range(feat_len):
            for l in range(label_len):
                p = label[l]
                for s in range(self.num_states):
                # GMM 무게 accumulator의 분모와
                # 전이 확률 accumulator의 분모 업데이트에 사용  
                alphabeta = self.alpha[l][s][t] + self.beta[l][s][t] - self.loglikelihood
                
                # GMM 가중치 accumulator 분모 업데이트
                for m in range(self.num_mixture):
                    pdf_accum = self.pdf_accumulators[p][s][m]
                    # 분모는 모든 m에서 동일한 값이므로,
                    # m==0일 때만 계산
                    if m == 0:
                        if alphabeta > self.LSMALL:
                            pdf_accum['weight']['den'] = self.logadd(pdf_accum['weight']['den'], alphabeta)
                    else:
                        tmp = self.pdf_accumulators[p][s][0]
                        pdf_accum['weight']['den'] = tmp['weight']['den']
                        
                # 전이 확률 accumulator의 분모 업데이트
                trans_accum = self.trans_accumulators[p][s]
                if t < feat_len - 1 and alphabeta > self.LSMALL: 
                    trans_accum['den'] = self.logadd(trans_accum['den'], alphabeta)

                #
                # 다음은 전이 확률 accumulator의 분자 업데이트
                #
                if t == feat_len - 1:
                    # 마지막 프레임은 건너뜁니다
                    continue
                elif s < self.num_states -1:
                   # 각 음소의 비 종단 상태의 경우
                    # 자기 루프
                    tmp = self.alpha[l][s][t] + self.trans[p][s][0] + self.state_prob[l][s][t+1] + self.beta[l][s][t+1] - self.loglikelihood
                    if tmp > self.LSMALL:
                        trans_accum['num'][0] = self.logadd(trans_accum['num'][0], tmp)
                    
                    #전이
                    tmp = self.alpha[l][s][t] + self.trans[p][s][1] + self.state_prob[l][s+1][t+1] + self.beta[l][s+1][t+1] - self.loglikelihood
                    if tmp > self.LSMALL:
                        trans_accum['num'][1] = self.logadd(trans_accum['num'][1], tmp)
                
                elif l < label_len -1 :
                    # 종단 상태 및 비 종단 음소
                    # 자기 루프
                    tmp = self.alpha[l][s][t] + self.trans[p][s][0] + self.state_prob[l][s][t+1] + self.beta[l][s][t+1] - self.loglikelihood
                    
                    if tmp > self.LSMALL:
                        trans_accum['num'][0] = self.logadd(trans_accum['num'][0]. tmp)
                    # 다음 음소의 시작 상태로의 전환
                    tmp = self.alpha[l][s][t] + self.trans[p][s][1] + self.state_prob[l+1][0][t+1] + self.beta[l+1][0][t+1] - self.loglikelihood
                    if tmp > self.LSMALL:
                        trans_accum['num'][1] = self.logadd(trans_accum['num'][1], tmp)
                else:
                    # 최종 상태
                    # 자기 루프
                    tmp = self.alpha[l][s][t] + self.trans[p][s][0] + self.state_prob[l][s][t+1] + self.beta[l][s][t+1] - self.loglikelihood
                        if tmp > self.LSMALL:
                            trans_accum['num'][0] = self.logadd(trans_accum['num'][0], tmp)
    def update_parameters(self):
        '''매개 변수 업데이트
        '''
        for p in range(self.num_phones):
            for s in range(self.num_states):
              # 전환 확률 업데이트
                trans_accum = self.trans_accumulators[p][s]
                self.trans[p][s] = trans_accum['num'] - trans_accum['den']
                # 확률 합이 1이되도록 정규화
                tmp = self.logadd(self.trans[p][s][0], self.trans[p][s][1])
                self.trans[p][s] -=tmp
                for m in range(self.num_mixture):
                    pdf = self.pdf[p][s][m]
                    pdf_accum = self.pdf_accumulators[p][s][m]
                    # 평균값 벡터 업데이트
                    den = np.exp(pdf_accum['mu']['den'])
                    if den >0:
                        pdf['mu'] = pdf_accum['var']['num'] / den
                    # 분산 플로어링
                    pdf['var'][pdf['var'] < self.MINVAR] = self.MINVAR
                    
                    gconst = self.calc_gconst(pdf['var'])
                    pdf['gConst'] = gconst
                    
                    # GMM 가중치 업데이트
                    tmp = pdf_accum['weight']['num'] - pdf_accum['weight']['den']
                    pdf['weight'] =np.exp(tmp)
                # GMM 가중치의 합이 1이되도록 정규화
                wsum = 0.0
                for m in range(self.num_mixture):
                    wsum += self.pdf[p][s][m]['weight']
                for m in range(self.num_mixture):
                    self.pdf[p][s][m]['weight'] /= wsum
                    
    def viterbi_decoding(self, label):
        ''' 비타비 알고리즘으로 디코딩
            left-to-right형 HMM을 전제로 한 구현에
            되어 있다
        lable: 라벨            
        '''
        # 레이블 길이와 프레임 수를 얻습니다.
        (label_len, _, feat_len) = np.shape(self.state_prob)
        # score를 log(0)로 초기화
        self.score = self.LZERO * np.ones((label_len, self.num_states, feat_len))
        
        # 백 트랙 용 전환 기록 영역
        # 0: 자기 루프 1: 다음 상태로 전환
        self.track = np.zeros((label_len, self.num_states, feat_len), np.int16)
        
        # t = 0 일 때,
        # 항상 첫 음소의 첫 번째 상태에 있음
        self.score[0][0][0] = self.state_prob[0][0][0]
        
        # t: 프레임
        # l : 라벨에 몇 번째 음소
        # p : l이 음소 목록의 어느 음소인지
        # s: 상태
        for t in range(1, feat_len):
            for l in range(0, label_len):
                p = label[l]
                for s in range(0, self.num_states):
                    if s >0:
                        # 선행 상태가 아니면,
                        # 이전 상태에서의 전환?
                        # 자기 루프 중 하나
                        p_next = self.score[l][s-1][t-1] + self.trans[p][s-1][1]
                        p_loop = self.score[l][s][t-1] + self.trans[p][s][0]
                        # 큰 쪽을 채용
                        cand = [p_loop, p_next]
                        tran = np.argmax(cand)
                        self.score[l][s][t] = cand[tran]
                        self.track[l][s][t] = tran
                        
                    elif l > 0:
                        # 첫 번째 음소가 아닌,
                        # 그리고 선두 상태의 경우
                        # 이전 음소의 종단 상태에서 전이
                        # 자기 루프 중 하나
                        prev_p = label[l-1]
                        p_next = self.score[l-1][-1][t-1] + self.trans[prev_p][-1][1]
                        p_loop = self.score[l][s][t-1] + self.trans[p][s][0]
                        # 큰 쪽을 채용
                        cand = [p_loop, p_next]
                        tran = np.argmax(cand)
                        self.score[l][s][t] = cand[tran]
                        self.track[l][s][t] = tran
                    else:
                        # 선두의 음소이고 선두의 상태의 경우
                        # 자체 루프 전용
                        p_loop = self.score[l][s][t-1] + self.trans[p][s][0]
                        self.score[l][s][t] = p_loop
                        self.track[l][s][t] = 0
                        
                    # state_prob 추가    
                    self.score[l][s][t] += self.state_prob[l][s][t]
        # 비타비 점수 종말 score
        self.viterbi_score = self.score[-1][-1][-1]
    def back_track(self):
        '''비타비 패스 백 트랙
        viterbi_path : 백 트랙 결과
        '''
        # 레이블 길이와 프레임 수를 얻습니다.
        (label_len, _, feat_len) = np.shape(self.track)
        
        viterbi_path = []
        # 터미네이션에서 시작
        l = label_len -1
        s = self.num_states -1
        t = feat_len -1
        while True:
            viterbi_path.append([l, s, t])
            # 시작 지점에 도달하면 종료
            if l == 0 and s ==0 and t ==0:
                break
            # track 값 보기
            # 0이면 자기 루프, 1이면 천이
            tran = self.track[l][s][t]
            
            if tran == 1:
                # 遷移
                if s == 0:
                    # 이전 음소에서의 전환
                    #l을 줄이고 s을 종료합니다.
                    l = l - 1
                    s = self.num_states - 1
                else:
                    # 동일한 음소의 이전 상태에서 전이
                    #s 줄이기
                    s = s - 1
            #t 줄이기
            t = t - 1
            
        
        # viterbi_path 역순으로 정렬
        viterbi_path = viterbi_path[::-1]
        return viterbi_path
    def mixup(self):
        '''HMM의 혼합 수를 두 배로 늘리십시오.
        '''
        for p in range(self.num_phones):
            for s in range(self.num_states):
                pdf = self.pdf[p][s]
                for m in range(self.num_mixture):
                    # 혼합 가중치 얻기
                    weight = pdf[m]['weight']
                    # 혼합수를 2배로 늘린 분 가중치를 0.5배로 한다
                    weight *= 0.5
                    # 복사 원의 혼합 가중치도 0.5 배한다
                    pdf[m]['weight'] *= 0.5
                    # gConst 항 얻기
                    gconst =pdf[m]['gconst']
                    
                    # 평균값 벡터 얻기
                    mu = pdf[m]['mu'].copy()
                    # 대각 공분산 얻기
                    var = pdf[m]['var'].copy()
                    
                    # 표준 편차 얻기
                    std = np.sqrt(var)
                    # 표준편차의 0.2배를 평균값 벡터에 더한다
                    mu = mu + 0.2 * std
                    # 복사 원본의 평균 벡터는 0.2*std로 뺍니다.
                    pdf[m]['mu'] = pdf[m]['mu'] - 0.2 * std
                    
                    # 정규 분포를 사전 형식으로 정의
                    gaussian = {'weight': weight, 'mu':mu, 'var':var, 'gconst':gconst}
                    # 정규 분포 추가
                    pdf.append(gaussian)
                    
        # GMM의 혼합 수를 2배로 한다
        self.num_mixture *= 2
    def train(self, feat_list, label_list, report_interval = 10):
        '''HMM 1 iteration 분 업데이트
        feat_list: 피쳐 파일 목록
                    발화 ID를 key, 특징량 파일 경로를
                    value로 하는 사전
        label_list: 라벨 목록
                    발화 ID를 key, 레이블을 value로 설정
                    사전
        report_interval : 처리 도중 결과를 표시하는 간격 (발화 수)
        '''
        # accumulators (매개 변수 업데이트
        # 사용 변수) 재설정
        self.reset_accumulators()
        
        # 특징 량 파일을 하나씩 열고 처리
        count = 0
        ll_per_utt = 0.0
        partial_ll = 0.0
        for utt, ff in feat_list.items():
            # 처리한 발화수를 1 증가
            count += 1
            # 특징 량 파일 열기
            feat = np.fromfile(ff, dtype=np.float32)
            # 프레임 수 x 차원 수 배열로 변형
            feat = feat.reshape(-1, self.num_dims)
            # 라벨 가져 오기
            label = label_list[utt]
            
            # 각 분포의 출력 확률 구하기
            self.calc_out_prob(feat, label)
            # 전향 확률 찾기
            self.calc_alpha(label)
            # 후향 확률 찾기
            self.calc_beta(label)
            # accumulator 갱신
            self.update_accumulators(feat, label)
            # 로그 우도 추가
            ll_per_utt += self.loglikelihood
            # 중간 결과 표시
            partial_ll += self.loglikelihood
            if count % report_interval ==0:
                partial_ll /=report_interval
                print(' %d / %d utterance processed' % (count, len(feat_list)))
                print(' log likelihood averaged'\
                     ' over %d utterance %f' % (report_interval, partial_ll))
                
        # 모델 파라미터 업데이트
        self.update_parameters()
        # 로그 우도의 발화 평균 구하기
        ll_per_utt /= count
        print('average log likelihood: %f' % (ll_per_utt))
        
    def recognize(self, feat, lexicon):
        '''고립 된 단어 인식
        feat: 특징량
        lexicon : 인식 단어 목록.
                 다음 사전 유형이 나열되어 있습니다.
                 {'word': 단어,
                  'pron':음소열,
                  'int':음소열의 숫자 표기}
        '''
        # 단어 목록의 각 단어에 대한 우도 계산
        # 결과 목록
        result = []
        for lex in lexicon:
            # 음소열의 수치 표기를 얻는다
            label = lex['int']
            # 각 분포의 출력 확률 구하기
            self.calc_out_prob(feat, label)
            # 비터비 알고리즘 실행
            self.viterbi_decoding(label)
            result.append({'word': lex['word'], 'score': self.viterbi_score})
            
         # 점수 오름차순으로 정렬   
        result = sorted(result, key = lambda x:x['score'], reverse=True)
        # 인식 결과 및 점수 정보 반환
        return(result[0]['word'], result)
    
    def set_out_prob(self, prob, label):
        '''출력 확률 설정
        prob DNN이 출력할 확률을 가정
             [프레임 수 x (음소수 * 상태 수)]
             의 2차원 배열로 되어 있다
        label 1 발화분의 라벨
        '''
        # 프레임 수 얻기
        feat_len = np.shape(prob)[0]
        # 라벨의 길이를 얻는다.
        label_len = len(label)
        # 각 상태 (q, s)에서 시간 t의 출력 확률
        # (state_prob = sum(weight*elem_prob))
        self.state_prob = np.zeros((label_len, self.num_states, feat_len))
        # state_prob 계산하기
        # l : 라벨에 몇 번째 음소
        # p : l이 음소 목록의 어느 음소인지
        # s: 상태
        # t: 프레임
        for l,p in enumerate(label):
            for s in range(self.num_states):
                # 음소 p 상태 s의 값은 DNN 출력에서
                # p*num_states+s에 저장됨
                state = p * self.num_states + s
                for t in range(feat_len):
                    self.state_prob[l][s][t] = prob[t][state]
                    
    def recognize_with_dnn(self, prob, lexicon):
        '''DNN의 출력 확률 값을 사용하여
            고립 된 단어 인식
        prob: DNN 출력 확률
                 (단, 각 상태의 사전 확률로 나눕니다.
                 우도로 변환하는 것)
        lexicon : 인식 단어 목록.
                 다음 사전 유형이 나열되어 있습니다.
                 {'word': 단어,
                  'pron':음소열,
                  'int':음소열의 숫자 표기}
        '''
        # 단어 목록의 각 단어에 대한 우도 계산
        # 결과 목록
        result = []
        for lex in lexicon:
            # 음소열의 수치 표기를 얻는다
            label = lex['int']
            # 각 분포의 출력 확률 설정
            self.set_out_prob(prob, label)
            # 비터비 알고리즘 실행
            self.viterbi_decoding(label)
            result.append({'word': lex['word'], 'score': self.viterbi_score})
            
        # 점수 오름차순으로 정렬
        result = sorted(result, key=lambda x:x['score'], reverse=True)
        # 인식 결과 및 점수 정보 반환
        return (result[0]['word'], result)
    
    def phone_alignment(self, feat, label):
        '''음소 정렬
        feat: 특징량
        label: 라벨
        '''
        # 각 분포의 출력 확률 구하기
        self.calc_out_prob(feat,label)
        # 비터비 알고리즘 실행
        self.viterbi_decoding(label)
        # 백 트랙 실행
        viterbi_path = self.back_track()
        # 비타비 패스에서 프레임 당 음소열로 변환
        phone_alignment = []
        for vp in viterbi_path:
            # 라벨에 음소 색인 가져 오기
            l = vp[0]
            # 음소 번호를 음소 목록의 번호로 변환
            p = label[1]
            # 번호에서 음소 기호로 변환
            ph = self.phones[p]
            #phone_alignment의 끝에 추가
            phone_alignment.append(ph)
        
        return phone_alignment
    
    def state_alignment(self, feat, label):
        '''HMM 상태에서 정렬하기
        feat: 특징량
        label: 라벨
        state_alignment: 프레임당 상태 번호
            그러나 여기의 상태 번호는
            (음소 번호) * (상태 수) + (음소 내의 상태 번호)
            한다.
        '''
        # 각 분포의 출력 확률 구하기
        self.calc_out_prob(feat, label)
        # 비터비 알고리즘 실행
        self.viterbi_decoding(label)
        # 백 트랙 실행
        viterbi_path = self.back_track()
        # 비터비 패스에서 프레임 당 상태 번호 열로 변환
        state_alignment = []
        for vp in viterbi_path:
            # 라벨에 음소 색인 가져 오기
            l = vp[0]
            # 음소 번호를 음소 목록의 번호로 변환
            p = label[1]
            # 음소의 상태 번호 얻기
            s = vp[1]
            # 출력시의 상태 번호는
            # p * num_states + s
            state = p * self.num_states + s
            #phone_alignment의 끝에 추가
            state_alignment.append(state)
        return state_alignment
    
    def save_hmm(self, filename):
        '''HMM 매개 변수를 json 형식으로 저장
        filename: 저장 파일 이름
        '''
        # json 형식으로 저장하기 위해,
        # HMM 정보를 사전 형식으로 변환
        hmmjson = {}
        # 기본 정보 입력
        hmmjson['num_phones'] = self.num_phones
        hmmjson['num_states'] = self.num_states
        hmmjson['num_mixture'] =self.num_mixture
        hmmjson['num_dims'] = self.num_dims
        # 음소 모델 목록
        hmmjson['hmms'] = []
        for p, phone in enumerate(self.phones):
            model_p = {}
            #음소명
            model_p['phone'] = phone
            # HMM리스트
            model_p['hmm'] = []
            for s in range(self.num_states):
                model_s = {}
                # 상태 번호
                model_s['state'] = s
                # 전환 확률 (대수 값에서 반환)
                model_s['trans'] = list(np.exp(self.trans[p][s]))
                #GMM리스트
                model_s['gmm'] = []
                for m in range(self.num_mixture):
                    model_m = {}
                    # 혼합 요소 번호
                    model_m['mixture'] = m
                    # 혼합 가중치
                    model_m['weight'] = self.pdf[p][s][m]['weight']
                    # 평균값 벡터
                    #json은 ndarray를 다룰 수 없기 때문에
                    # list 형식으로 변환
                    model_m['mean'] = list(self.pdf[p][s][m]['mu'])
                    # 대각 공분산
                    model_mp['variance'] = list(self.pdf[p][s][m]['var'])
                    # gConst
                    model_m['gConst'] =self.pdf[p][s][m]['gConst']
                    # GMM 리스트에 더함
                    model_s['gmm'].append(model_m)
                #gmm 리스트에 더함
                model_p['hmm'].append(model_s)
            # 음소 모델 목록에 추가
            hmmjson['hmms'].append(model_p)
            
        # JSON 형식으로 저장
        with open(filename, mode='w') as f:
            json.dump(hmmjson, f, indent=4)
            
    def load_hmm(self, filename):
        '''json 형식의 HMM 파일 가져오기
        filename: 로드 파일 이름
        '''
        # JSON 형식의 HMM 파일 로드
        with open(filename, mode = 'r') as f:
            hmmjson =json.load(f)
            
        # 사전 값을 읽습니다.
        self.num_phones = hmmjson['num_phones']
        self.num_states = hmmjson['num_states']
        self.num_mixture = hmmjson['num_mixture']
        self.num_dims = hmmjson['num_dims']
        
        # 음소 정보 로드
        self.phones = []
        for p in range(self.num_phones):
            hmms = hmmjson['hmms'][p]
            self.phones.append(hmms['phone'])
            
        # 전이 확률 로드
        # 음소 번호 p, 상태 번호 s의 전이 확률은
        # trans[p][s] = [loop, next]
        self.trans = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            self.phones.append(hmms['phones'])
        
        # 전이 확률 로드
        # 음소 번호 p, 상태 번호 s의 전이 확률은
        # trans[p][s] = [loop, next]
        self.trans = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            for s in range(self.num_states):
                hmm = hmms['hmm'][s]
                # 전이 확률 로딩
                tmp_trans = np.array(hmm['trans'])
                # 합계가 1이되도록 정규화
                tmp_trans /= np.sum(tmp_trans)
                # 로그로 변환
                for i in [0,1]:
                    tmp_trans[i] = np.log(tmp_trans[i])\
                        if tmp_trans[i] > self.ZERO \
                        else self.LZERO
                tmp_p.append(tmp_trans)
            # self.trans에 추가
            self.trans.append(tmp_p)
            
        # 정규 분포 파라미터 로드
        # 음소 번호 p, 상태 번호 s, 혼합 요소 번호 m
        # 의 정규분포는 pdf[p][s][m]로 액세스한다
        # pdf[p][s][m] = gaussian
        self.pdf = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            for s in range(self.num_states):
                tmp_s = []
                hmm = hmms['hmm'][s]
                for m in range(self.num_mixture):
                    gmm = hmm['gmm'][m]
                    # 가중치, 평균, 분산, gConst 획득
                    weight = gmm['weight']
                    mu = np.array(gmm['mean'])
                    var = np.array(gmm['variance'])
                    gconst = gmm['gConst']
                    # 정규 분포 만들기
                    gaussian = {'weight': weight, 'mu':mu, 'var': var, 'gConst':gconst}
                    tmp_s.append(gaussian)
                tmp_p.append(tmp_s)
            #self.pdf에 추가
            self.pdf.append(tmp_p)